In [1]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.optimize as opt
import math

from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyarrow.csv as csv
import pyarrow.json as json

from enum import Enum
import abc
from typing import List, Dict, Tuple, Any, Union

In [2]:
sys.path.append('/groups/icecube/cyan/Utils')
from PlotUtils import setMplParam, getColour, getHistoParam 
# getHistoParam:
# Nbins, binwidth, bins, counts, bin_centers = 
from DB_lister import list_content, list_tables
from ExternalFunctions import nice_string_output, add_text_to_ax
setMplParam()

In [3]:
sys.path.append('/groups/icecube/cyan/factory/DOMification')
from Enum.Flavour import Flavour
from Enum.EnergyRange import EnergyRange

In [40]:
root_dir = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered_second_round/Snowstorm/CC_CRclean_IntraTravelDistance_250/"
root_dir_32 = root_dir + "22032/"
dir_32_1 = root_dir_32 + "1/"
truth_32_1 = root_dir_32 + "truth_1.parquet"
pmt_32_1_1 = dir_32_1 + "PMTfied_1.parquet"
pmt_32_1_2 = dir_32_1 + "PMTfied_2.parquet"

root_dir_33 = root_dir + "22033/"
dir_33_1 = root_dir_33 + "1/"
truth_33_1 = root_dir_33 + "truth_1.parquet"
pmt_33_1_1 = dir_33_1 + "PMTfied_1.parquet"
pmt_33_1_2 = dir_33_1 + "PMTfied_2.parquet"

dir_33_5 = root_dir_33 + "5/"
truth_33_5 = root_dir_33 + "truth_5.parquet"
pmt_33_5_1 = dir_33_5 + "PMTfied_1.parquet"
pmt_33_5_2 = dir_33_5 + "PMTfied_2.parquet"

dir_33_6 = root_dir_33 + "6/"
truth_33_6 = root_dir_33 + "truth_6.parquet"
pmt_33_6_1 = dir_33_6 + "PMTfied_1.parquet"
pmt_33_6_2 = dir_33_6 + "PMTfied_2.parquet"

N_events_per_part = 30000  
N_events_per_shard = 3000  
n_combined  = min(1PeV-100PeV, 100TeV-1PeV) * 2
| Flavour | 100TeV-1PeV | 1PeV-100PeV | total   | n_combined | n_parts | n_shards |
|---------|-------------|-------------|---------|-----------|-----------|-----|
| νₑ      | 27,498      | 24.857      | 52,355  | 49,714    | 49,714 // 30,000 = 2 | 49,714 // 3000 = 17 |
| νμ      | 165,226     | 96,138      | 261,364 | 192,276   | 192,276 // 30,000 = 6 | 192,276 // 3000 = 64 |
| ντ      | 31,473      | 30.143      | 51,536  | 60,286    | 60,286 // 30,000 = 2 | 60,286 // 3000 = 20 |

In [18]:
def convertParquetToDF(file:str) -> pd.DataFrame:
    table = pq.read_table(file)
    df = table.to_pandas()
    return df

In [19]:
def show_offset_edges(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identifies rows where the `offset` decreases or stays the same compared to the previous row.
    Useful for checking whether offset resets appropriately or has unexpected drops.
    """
    df = df.reset_index(drop=True)
    offset_diff = df['offset'].diff()
    mask = offset_diff < 0
    return df.loc[mask]

def show_shard_edges(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identifies the rows where `shard_no` changes.
    Helpful to observe shard boundaries and investigate the offset reset logic.
    """
    df = df.reset_index(drop=True)
    shard_change = df['shard_no'].ne(df['shard_no'].shift())
    return df.loc[shard_change]


In [20]:
df_truth_32_1 = convertParquetToDF(truth_32_1)
df_pmt_32_1_1 = convertParquetToDF(pmt_32_1_1)
df_pmt_32_1_2 = convertParquetToDF(pmt_32_1_2)

In [21]:
df_truth_32_1

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,114000400665696,665696,32,1,1,170,170,1.312122e+05,0.045589,2.148992,...,1.844175e+06,1.0,2.700705e-05,8.891057e-35,1286.661743,7780.790527,0.5,456.220154,0,1018.384155
1,115000600138754,138754,32,1,1,93,263,2.677757e+06,3.056381,1.249584,...,1.828938e+05,1.0,1.629861e-04,1.490586e-33,1012.475159,972.081787,0.5,681.626709,0,1059.699829
2,114000400665707,665707,32,1,1,151,414,2.835897e+05,5.323183,1.584131,...,1.465076e+05,1.0,4.410268e-05,5.035121e-34,799.276367,790.458862,0.5,474.023773,1,483.897614
3,115000600138761,138761,32,1,1,893,1307,5.884352e+06,0.392921,1.501227,...,1.748610e+05,1.0,2.209616e-04,2.113741e-33,961.688782,935.705811,0.5,987.917358,1,875.515137
4,114000400665728,665728,32,1,1,215,1522,3.105628e+05,4.643663,2.557549,...,2.281205e+05,1.0,1.880685e-05,5.151188e-34,1215.146362,1062.376099,0.5,672.177246,0,1158.643921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,115001200321016,321016,32,1,10,11,1006271,2.574150e+06,2.170059,2.192222,...,2.153553e+05,1.0,1.136798e-05,1.183444e-33,1126.857178,1209.935425,0.5,186.121750,0,1326.732544
29996,114000100005891,5891,32,1,10,60,1006331,4.040711e+05,1.399143,2.032368,...,1.735588e+05,1.0,1.440617e-05,5.169753e-34,1050.716675,832.547668,0.5,497.447296,0,730.485474
29997,115001200321023,321023,32,1,10,106,1006437,8.741097e+07,1.544273,2.548269,...,2.295298e+05,1.0,2.695232e-12,1.213904e-33,1059.422974,1232.254150,0.5,583.592896,0,411.081970
29998,114000100005903,5903,32,1,10,214,1006651,5.598987e+05,0.150352,2.100199,...,3.357129e+06,1.0,1.379161e-03,6.880518e-34,765.327576,13084.187500,0.5,979.527954,0,504.981689


In [22]:
df_truth_32_1['pid']

0       -12
1        12
2        12
3        12
4        12
         ..
29995    12
29996    12
29997    12
29998   -12
29999    12
Name: pid, Length: 30000, dtype: int64

In [23]:
show_offset_edges(df_truth_32_1)

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
3000,114000400688920,688920,32,1,2,195,195,280606.968750,1.047351,1.644178,...,155116.500000,1.0,4.643797e-05,5.007529e-34,860.239136,822.909973,0.5,626.275269,1,272.935181
6000,114000400711049,711049,32,1,3,100,100,407463.468750,2.471314,1.979179,...,171071.843750,1.0,6.196543e-05,6.058632e-34,1078.285522,777.992981,0.5,500.795593,0,687.211365
9000,114000400734908,734908,32,1,4,89,89,404612.093750,4.690163,2.551369,...,229059.734375,1.0,7.756918e-07,2.934428e-34,1137.805298,1149.102783,0.5,587.942200,0,1248.589233
12000,114000400758290,758290,32,1,5,384,384,187251.468750,1.502958,1.654667,...,174423.625000,1.0,4.214020e-05,4.041052e-34,928.330627,964.317749,0.5,801.923645,1,840.070679
15000,114000400781242,781242,32,1,6,689,689,130601.375000,0.844760,2.338582,...,209308.015625,1.0,4.149801e-05,3.316248e-34,992.102051,1041.493286,0.5,901.353088,1,675.029724
18000,114000400803627,803627,32,1,7,930,930,302175.437500,2.488098,0.344550,...,202193.140625,1.0,6.286679e-05,5.200779e-34,984.515625,1034.124634,0.5,996.703613,1,282.800110
21000,114000400826672,826672,32,1,8,99,99,220246.796875,2.726448,0.345144,...,202233.328125,1.0,5.330473e-05,4.408760e-34,1027.434326,991.636902,0.5,528.049255,1,947.936646
24000,114000400849928,849928,32,1,9,213,213,105879.031250,1.647035,2.093771,...,215560.906250,1.0,7.104204e-06,2.090847e-34,803.708740,1235.755249,0.5,820.528137,0,645.511292
27000,114000400872883,872883,32,1,10,387,387,759809.500000,5.902627,1.843117,...,178250.156250,1.0,8.797578e-05,8.255660e-34,979.916687,954.252808,0.5,781.120117,1,413.731171


In [24]:
show_shard_edges(df_truth_32_1)

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,114000400665696,665696,32,1,1,170,170,131212.156250,0.045589,2.148992,...,1.844175e+06,1.0,2.700705e-05,8.891057e-35,1286.661743,7780.790527,0.5,456.220154,0,1018.384155
3000,114000400688920,688920,32,1,2,195,195,280606.968750,1.047351,1.644178,...,1.551165e+05,1.0,4.643797e-05,5.007529e-34,860.239136,822.909973,0.5,626.275269,1,272.935181
6000,114000400711049,711049,32,1,3,100,100,407463.468750,2.471314,1.979179,...,1.710718e+05,1.0,6.196543e-05,6.058632e-34,1078.285522,777.992981,0.5,500.795593,0,687.211365
9000,114000400734908,734908,32,1,4,89,89,404612.093750,4.690163,2.551369,...,2.290597e+05,1.0,7.756918e-07,2.934428e-34,1137.805298,1149.102783,0.5,587.942200,0,1248.589233
12000,114000400758290,758290,32,1,5,384,384,187251.468750,1.502958,1.654667,...,1.744236e+05,1.0,4.214020e-05,4.041052e-34,928.330627,964.317749,0.5,801.923645,1,840.070679
15000,114000400781242,781242,32,1,6,689,689,130601.375000,0.844760,2.338582,...,2.093080e+05,1.0,4.149801e-05,3.316248e-34,992.102051,1041.493286,0.5,901.353088,1,675.029724
18000,114000400803627,803627,32,1,7,930,930,302175.437500,2.488098,0.344550,...,2.021931e+05,1.0,6.286679e-05,5.200779e-34,984.515625,1034.124634,0.5,996.703613,1,282.800110
21000,114000400826672,826672,32,1,8,99,99,220246.796875,2.726448,0.345144,...,2.022333e+05,1.0,5.330473e-05,4.408760e-34,1027.434326,991.636902,0.5,528.049255,1,947.936646
24000,114000400849928,849928,32,1,9,213,213,105879.031250,1.647035,2.093771,...,2.155609e+05,1.0,7.104204e-06,2.090847e-34,803.708740,1235.755249,0.5,820.528137,0,645.511292
27000,114000400872883,872883,32,1,10,387,387,759809.500000,5.902627,1.843117,...,1.782502e+05,1.0,8.797578e-05,8.255660e-34,979.916687,954.252808,0.5,781.120117,1,413.731171


In [25]:
df_pmt_32_1_1['event_no'].unique()

array([114000400665696, 115000600138754, 114000400665707, ...,
       115000600145826, 114000400688913, 115000600145827])

In [26]:
df_pmt_32_1_1

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,t2,t3,t4,t5,T10,T50,sigmaT,eccentricity_PCA,aspect_contrast_PCA,hypotenuse
0,114000400665696,665696,79.410004,-248.240005,277.630005,-18.810898,-170.445206,88.687332,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,1.000000,42.474106
1,114000400665696,665696,79.410004,-248.240005,260.609985,-18.810898,-170.445206,71.667313,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,1.000000,42.474106
2,114000400665696,665696,79.410004,-248.240005,141.470001,-18.810898,-170.445206,-47.472672,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,1.000000,42.474106
3,114000400665696,665696,210.470001,-209.770004,360.779999,112.249100,-131.975204,171.837326,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,1.000000,42.474106
4,114000400665696,665696,210.470001,-209.770004,292.700012,112.249100,-131.975204,103.757339,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,1.000000,42.474106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985323,115000600145827,145827,-481.600006,101.389999,-348.980011,-268.972595,610.276672,-808.066650,0.0444,1.0,...,13862.0,-1.0,-1.0,-1.0,0.0,2.0,1.0,0.901014,0.394904,371.898895
985324,115000600145827,145827,-481.600006,101.389999,-366.000000,-268.972595,610.276672,-825.086670,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.901014,0.394904,371.898895
985325,115000600145827,145827,-481.600006,101.389999,-383.019989,-268.972595,610.276672,-842.106689,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.901014,0.394904,371.898895
985326,115000600145827,145827,-358.440002,120.559998,-404.040009,-145.812576,629.446655,-863.126709,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.901014,0.394904,371.898895


In [27]:
df_pmt_32_1_1['event_no'].unique()

array([114000400665696, 115000600138754, 114000400665707, ...,
       115000600145826, 114000400688913, 115000600145827])

In [28]:
df_pmt_32_1_2

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,t2,t3,t4,t5,T10,T50,sigmaT,eccentricity_PCA,aspect_contrast_PCA,hypotenuse
0,114000400688920,688920,-570.900024,-125.139999,346.420013,-159.321136,-301.747314,71.550781,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.815767,0.267121,264.954285
1,114000400688920,688920,-447.739990,-113.129997,313.450012,-36.161102,-289.737305,38.580780,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.815767,0.267121,264.954285
2,114000400688920,688920,-447.739990,-113.129997,296.429993,-36.161102,-289.737305,21.560760,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.815767,0.267121,264.954285
3,114000400688920,688920,-447.739990,-113.129997,228.339996,-36.161102,-289.737305,-46.529236,0.0444,1.0,...,12127.0,-1.0,-1.0,-1.0,0.0,12.0,6.000000,0.815767,0.267121,264.954285
4,114000400688920,688920,-324.390015,-93.430000,312.910004,87.188873,-270.037292,38.040771,0.0444,1.0,...,11853.0,-1.0,-1.0,-1.0,0.0,655.0,327.500000,0.815767,0.267121,264.954285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002988,115000600153071,153071,-224.089996,470.859985,468.380005,102.776978,19.576904,16.960907,0.0444,1.0,...,10671.0,11050.0,11569.0,-1.0,0.0,570.0,415.835602,-1.000000,-1.000000,-1.000000
1002989,115000600153071,153071,-224.089996,470.859985,451.359985,102.776978,19.576904,-0.059113,0.0444,1.0,...,11226.0,12039.0,-1.0,-1.0,0.0,634.0,592.240051,-1.000000,-1.000000,-1.000000
1002990,115000600153071,153071,-224.089996,470.859985,434.339996,102.776978,19.576904,-17.079102,0.0444,1.0,...,12458.0,-1.0,-1.0,-1.0,0.0,0.0,311.500000,-1.000000,-1.000000,-1.000000
1002991,115000600153071,153071,-224.089996,470.859985,417.320007,102.776978,19.576904,-34.099091,0.0444,1.0,...,12119.0,-1.0,-1.0,-1.0,0.0,0.0,376.500000,-1.000000,-1.000000,-1.000000


In [29]:
df_pmt_32_1_2['event_no'].nunique()

3000

In [30]:
df_truth_33_1 = convertParquetToDF(truth_33_1)
df_pmt_33_1_1 = convertParquetToDF(pmt_33_1_1)
df_pmt_33_1_2 = convertParquetToDF(pmt_33_1_2)

In [31]:
df_truth_33_1

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,117000100000000,0,33,1,1,29,29,2.202673e+05,1.057394,1.096543,...,4.579077e+05,1.0,0.000121,4.408979e-34,937.037842,4106.984863,0.5,807.275879,0,842.590698
1,118000200034503,34503,33,1,1,1288,1317,9.993943e+07,4.236219,1.008906,...,4.781999e+05,1.0,0.001907,6.672074e-33,1219.984375,4033.478271,0.5,1415.224365,0,878.541992
2,117000100000016,16,33,1,1,117,1434,1.030255e+05,5.141798,1.164492,...,5.862259e+05,1.0,0.000086,2.451580e-34,1109.471191,5338.586914,0.5,545.214417,0,1101.875366
3,118000200034504,34504,33,1,1,285,1719,5.813970e+06,5.110892,2.227267,...,2.880898e+06,1.0,0.000666,3.866786e-34,926.941895,11231.840820,0.5,1195.848877,0,769.854675
4,117000100000023,23,33,1,1,164,1883,1.417055e+05,2.240111,1.921963,...,2.647948e+06,1.0,0.000477,3.011335e-34,1003.943298,11241.443359,0.5,1230.006836,1,1036.770630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,118000600190103,190103,33,1,10,106,938006,1.359993e+06,0.829631,1.021416,...,4.457075e+05,1.0,0.000281,1.054004e-33,1121.664673,3780.543213,0.5,516.010010,0,1279.324341
29996,117000100181640,181640,33,1,10,99,938105,1.015953e+05,1.800431,2.283835,...,2.507544e+06,1.0,0.000431,2.876451e-34,1207.534424,10061.763672,0.5,533.201721,0,1013.937073
29997,118000600190104,190104,33,1,10,190,938295,1.808180e+06,4.202335,1.780290,...,3.965734e+06,1.0,0.002865,1.211454e-33,1017.872070,18048.265625,0.5,1102.203125,0,1105.729248
29998,117000100181647,181647,33,1,10,61,938356,1.241591e+05,0.581222,2.509198,...,2.559200e+06,1.0,0.000423,2.764717e-34,820.273071,10302.372070,0.5,752.082214,0,588.759521


In [45]:
df_truth_33_1['event_no']

0        117000100000000
1        118000200034503
2        117000100000016
3        118000200034504
4        117000100000023
              ...       
29995    118000600190103
29996    117000100181640
29997    118000600190104
29998    117000100181647
29999    118000600190105
Name: event_no, Length: 30000, dtype: int64

In [35]:
df_pmt_33_1_1['event_no'].unique()

array([117000100000000, 118000200034503, 117000100000016, ...,
       118000200039828, 117000100018481, 118000200039831])

In [38]:
df_truth_33_5 = convertParquetToDF(truth_33_5)
df_pmt_33_5_1 = convertParquetToDF(pmt_33_5_1)
df_pmt_33_5_2 = convertParquetToDF(pmt_33_5_2)

In [39]:
df_truth_33_5

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,117000200390565,390565,33,5,1,197,197,1.524592e+05,4.292608,0.103457,...,2.806119e+05,1.0,0.000053,3.149408e-34,944.820007,1968.749756,0.5,593.003479,1,513.074036
1,118000400109950,109950,33,5,1,395,592,3.207414e+06,2.921350,0.758462,...,3.215911e+05,1.0,0.000305,1.586627e-33,1063.643555,2473.319580,0.5,1292.646851,0,981.217285
2,117000200390574,390574,33,5,1,27,619,9.494376e+05,1.700950,1.593053,...,3.591866e+06,1.0,0.001971,9.194248e-34,928.731018,38046.136719,0.5,650.961914,0,975.446533
3,118000400109955,109955,33,5,1,579,1198,6.430967e+07,5.776492,1.433426,...,1.152492e+06,1.0,0.003866,5.630251e-33,852.565186,11901.824219,0.5,1059.763428,0,787.240845
4,117000200390581,390581,33,5,1,38,1236,2.253646e+05,4.356098,2.732615,...,8.422606e+05,1.0,0.000018,1.817420e-35,1089.861450,3376.559082,0.5,389.340637,0,1130.310425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,118000800266453,266453,33,5,10,720,1006632,2.138830e+06,0.456322,1.011055,...,5.254348e+05,1.0,0.000412,1.312865e-33,899.898865,4555.517090,0.5,1036.896851,0,381.915802
29996,117000200570682,570682,33,5,10,23,1006655,1.813019e+05,0.494717,0.718237,...,3.706282e+05,1.0,0.000088,3.972451e-34,798.161560,3050.006348,0.5,354.056000,0,422.797119
29997,118000800266454,266454,33,5,10,189,1006844,2.603362e+06,4.255904,2.868094,...,2.432052e+06,1.0,0.000323,2.219502e-34,1134.834961,9270.193359,0.5,645.756348,0,421.682495
29998,117000200570699,570699,33,5,10,135,1006979,2.140610e+05,2.562129,0.891945,...,4.421217e+05,1.0,0.000102,3.875765e-34,1331.401001,3274.429932,0.5,576.844727,1,1143.459106


In [46]:
df_truth_33_5['event_no']

0        117000200390565
1        118000400109950
2        117000200390574
3        118000400109955
4        117000200390581
              ...       
29995    118000800266453
29996    117000200570682
29997    118000800266454
29998    117000200570699
29999    118000800266455
Name: event_no, Length: 30000, dtype: int64

In [43]:
df_pmt_33_5_1['event_no'].unique()

array([117000200390565, 118000400109950, 117000200390574, ...,
       118000400115169, 117000200408432, 118000400115174])

In [44]:
df_pmt_33_5_2['event_no'].unique()

array([117000200408448, 118000400115177, 117000200408449, ...,
       118000400120428, 117000200426073, 118000400120431])

In [41]:
df_truth_33_6 = convertParquetToDF(truth_33_6)
df_pmt_33_6_1 = convertParquetToDF(pmt_33_6_1)
df_pmt_33_6_2 = convertParquetToDF(pmt_33_6_2)

In [42]:
df_truth_33_6

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,117000200570712,570712,33,6,1,141,141,6.125652e+05,5.833509,1.394848,...,1.064684e+06,1.0,0.000446,6.999744e-34,952.735962,10795.324219,0.5,1003.116577,0,988.894531
1,118000800266462,266462,33,6,1,516,657,1.433632e+06,5.732343,0.960404,...,4.151947e+05,1.0,0.000278,1.118179e-33,1134.586792,3430.672363,0.5,796.576965,1,257.736816
2,117000200570715,570715,33,6,1,255,912,6.304251e+05,4.114387,1.184500,...,5.205431e+05,1.0,0.000221,7.102842e-34,956.202209,4783.405273,0.5,1279.205200,0,1194.334961
3,118000800266463,266463,33,6,1,556,1468,4.541098e+07,2.365034,1.346689,...,8.703636e+05,1.0,0.002553,4.913893e-33,959.312195,8639.135742,0.5,1266.803955,0,988.022705
4,117000200570720,570720,33,6,1,176,1644,3.729766e+05,0.972789,0.790076,...,3.965238e+05,1.0,0.000137,5.796111e-34,1264.152344,2853.036865,0.5,1292.152344,0,737.176208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4625,118000800274764,274764,33,6,2,246,518239,8.912645e+06,0.325375,1.815984,...,1.733376e+05,1.0,0.000161,7.785092e-34,877.530701,1003.333069,0.5,701.336853,0,896.242004
4626,117000200598725,598725,33,6,2,18,518257,1.018276e+05,3.674282,2.828493,...,5.762369e+05,1.0,0.000003,4.842290e-36,1141.969849,2274.752930,0.5,742.969177,0,399.936279
4627,118000800274765,274765,33,6,2,112,518369,6.767798e+06,0.787862,1.085564,...,3.545912e+05,1.0,0.000476,2.245841e-33,748.337830,3173.055176,0.5,620.778931,0,322.927734
4628,117000200598747,598747,33,6,2,608,518977,3.554336e+05,1.393071,1.300711,...,8.262652e+05,1.0,0.000279,5.656965e-34,1038.089722,8056.224121,0.5,799.624390,1,865.916992
